1.
The goal of this project is to be able to create a machine learning classificiation model using selected financial and email features that is able to identify the person of interest that is involved in the Enrol fraud scandal.
The data set contains of 146 rows which consists of 145 people and 1 row summing the total number for all the features. There are 18 POIs (Person of Interests) in the data set (around 12.4% of the whole data). Person of interest is defined as someone who is involved in the fraud by being an accomplice or an actual perpetrator and is labeled as 1 in the data (0 is for innocent people working at Enron during that time). These numbers are actually on the low side for a typical machine learning task: we have very few data points and even fewer people who are labeled as the culprits. This task is also quite unique because we are not trying to predict future data but we are just trying to create an algorithm that is able to identify these POIs for us using past data that are avaiable for us.

First, I decided to count the number of NaNs for each feature and found out that there are a few features that has high number of NaNs (>70% of NaNs), which are: loan_advancements, deferral_payments, director_fees and restristed_stock_deferred.

``` python
#Check how many missing values we have per feature
missing_values = dict()

for i in range(0,len(features_list)):
    missing_values[features_list[i]]=0
    for key in data_dict.keys():
        if data_dict[key][features_list[i]]=='NaN':
            missing_values[features_list[i]]+=1
            
print missing_values
```

I decided to remove loan_advancements since we only have 4 data points for that feature but keep the rest as the absence of these features (meaning having 0 as a value) might be an indicator of being innocent (and since we have an imbalance ratio of POIs and non-POIs so NaN can be a good indicator). Also, I plotted the financial features against each other in order to get a sense of the data and there are some outliers especially for the stocks. 

``` python
### Get feel of features to check outliers
import matplotlib.pyplot
for i in range(1,len(features_list)):
    for j in range(1,len(features_list)):
        if j>i:
            for point in plot_data:
                if point[0]==0:
                    matplotlib.pyplot.scatter( point[i], point[j],c='b' )
                if point[0]==1:
                    matplotlib.pyplot.scatter(point[i],point[j],c='r')
            matplotlib.pyplot.xlabel(features_list[i])
            matplotlib.pyplot.ylabel(features_list[j])
            matplotlib.pyplot.show()
```

I decided to keep it because the fact of being an outlier financially can be a good indicator of being a culprit since they are actually the people who made a lot of money from this fraud and I have found that removing the outlier dropped my recall and precision score slightly. From the plot, I also notice there is one data point that is much higher than every other data points consistently. After investigating, I found out that apparently, the row is the "TOTAL" of everyone in the data set for each removed so I decided to remove the outlier.

``` python
# remove "Total" outlier
#Total row has the highest salary
current=0
for key in data_dict.keys():  
    if data_dict[key]['salary']>current and data_dict[key]['salary']!='NaN':
        total_outlier=key
        current=data_dict[key]['salary']
    
print total_outlier,current
del data_dict[total_outlier]
```

2.
I ended up using: salary, deferral_payments, total_payments, bonus, deferred_income, total_stock_value, expenses, exercised_stock_options, long_term_incentive, restricted_stock, director_fees and 3 other features for email that I made which are basically fractions of email to/from POI over the total number to/from emails: to_poi_frac, from_poi_frac and shared_recepient_frac. I removed director_fees and restricted_stock_deferred because not only they have high number of missing values, the data that we have for those features are only for non-POIs, which I checked using this code:

``` python
#Create DataFrame with count of 0s and 1s for each feature with missing value
import pandas as pd

data = featureFormat(data_dict, features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)

nans = pd.DataFrame(0, index=[0,1], columns=features_list[1:])
for i in range(len(features)):
    label = int(labels[i])
    for j in range(len(features[i])):
        if features[i][j] == 0.0:
            nans.iloc[label,j] += 1
        
nans.T
```

I created the fraction of email features as suggested from the tutorial videos and it makes more sense to reduce 5 features into 3 simpler features that gives a better estimate of the interaction of the person with POIs. I also removed other because I do not understand the meaning of the 'other' feature. I was also able to rank feature importances because I was using a decision tree classifier as the base estimator for my ADA Boost.

``` python
importances = clf.feature_importances_
import numpy as np
indices = np.argsort(importances)[::-1]
print 'Feature Ranking: '
for i in range(10):
    print "{} feature no.{} ({})".format(i+1,indices[i],importances[indices[i]]) 
print features_list
```

The old email features ('to_messages', 'from_poi_to_this_person', 'from_messages', 'from_this_person_to_poi', 'shared_receipt_with_poi')
has features importance of (0.0133, 0.0133, 0.0133, 0.0667, 0.08) respectively with a total of 0.187. The new features that were created from these old email features ('to_poi_frac', 'from_poi_frac', 'shared_recipient_frac') has features importance of (0.08, 0.02666667,  0.02666667) with total of 0.133. Even though from the importnaces, my newly made features about email interaction with POIs have lower importance than the initial email features, I still decide to use them because I am substituing 5 features to just 3. And the email features related to POI has potential to create data leakage and bias to the model since it contains information of the poi itself, but since the feature importance is low, this signifies that the bias is not that strong and financial features are much more important than these POI email features. 

Looking at the rankings, I decided to use all the remaining financial features as it seems to contain information that contributes the model. I tried different combinations of financial features and got the best results when I used all of them, but I decided to remove director_fees since it has very small effect. Below is the list of recall and precision values I got when using the different combinations:
1. Top 5 financial features (exercised_stock_options, salary, restricted_stock, total_stock_value, expenses): recall = 0.294, precision = 0.326
2. Using stock related only financial features: recall = 0.257, precision = 0.353
3. Using non stock related only financial features: recall = 0.319 , precision = 0.364
4. Using all financial features: recall = 0.319 , precision = 0.39
5. Using all financial features except director_fees: recall = 0.318 , precision = 0.39

Finally, I did not scale my features because I used a decision tree classifier that does not require feature scaling unlike SVMs.


3.
After trying out different parameters mainly GaussianNB, RandomForest and AdaBoost, I have decided to use AdaBoost at the end. I found that GaussianNB to be not robust, by simply removing one feature, the resulting accuracy, precision and recall can be really different. Random Forests are much more robusts but results in overfitting. AdaBoost gave me the best results at the end.

``` python
#Past models tried
clf = GaussianNB()
clf=tree.DecisionTreeClassifier(random_state = 1)
clf=svm.SVC()
clf = RandomForestClassifier(n_estimators=10, random_state=1, verbose=0)
```


4.
I initially tuned the parameters of Random Forest but after using GridSearchCV to test out n_estimators as well as max depth, I got really high scores for my train_set but really low results for my test_set. This implies that I have a problem with overfitting when using random forests. Then, I used ADA Boost with DecisionTreeClassifier as the base estimators. Again using GridSearchCV with n_estimators and learning rate, I found that learning rate of 1 is the optimal and by increasing the n_estimators above 50, I was able to achieve past the minimum requirement of 0.3 recall and precision but there is a tradeoff with the time of fitting and predicting if I use a higher number of estimators. 

``` python
#GridSearchCV for AdaBoostClassfier
parameters = {'learning_rate':[0.01, 0.1, 0.5, 1],'n_estimators':[25,50,75,100]}
model = AdaBoostClassifier(random_state=9)

gs = GridSearchCV(model, parameters, verbose = 2, scoring='recall',
                   cv= StratifiedShuffleSplit(n_splits=100, random_state=9) ,n_jobs = 3)
gs.fit(features, labels)
gs.best_params_
gs.cv_results_

# And finally we get:
clf = AdaBoostClassifier(random_state=7, n_estimators=75, learning_rate=1)
```

Parameter tuning is really important because with my AdaBoost, using low learning rate or higher learning rate will result in very drastic change in the precision recall. Learning rate is how much consideration the model should take from each base estimator, a low learning rate will result in over fitting as it takes a lot of consideration for each tree used as base estimator while a high learning rate will result in underfitting because it ignores a lot of information provided by the base estimators. The number of estimators used in AdaBoost is also really important as it determines how many estimators you want to use to create the model and will strongly correlate with the time taken to fit and predict using that model.


5.
Validation is creating train and test data from our existing data to train and test the model that we created and check its performance. The reason we should split our data into test and train is to estimate the performance of our model for an independent data set and check for over fitting. If we use all our data to train, and then use the same data to test, this will result in overfitting since the model uses the same data set to fit and to create predictions and the high scores we get does not reflect the true ability of the model. Even though our case is unique since we are cannot get any more future data, we still want to create a model that can actually predict the POIs, so we should split up the data set into train and test data.

A classic tradeoff is the ratio between train data and test data that we obtained when we split our initial data set. Using a lot more train data will give us a better model but it will result in too little data to test with. Also since there is an imbalance in amount of POIs and non-POIs in our data set, we need to split the data more evenly between train and test so it has a balanced amount of POIs and non-POIs in the train and test data. Also, we need to randomize the split between test and train data to prevent any bias created with using a certain train and test data. This can be achieved by using the StratifiedShuffleSplit from sklearn as this function allows us to split the POIs and non-POIs between our train and test data with equal ratio as with the whole data set. It also allows us to randomly split the data while maintaining this ratio and perform the fitting and testing of the model multiple times (in this case, it was performed 1000 times) and then average the performance and results of the predicitions of the model.

``` python
from sklearn.cross_validation import StratifiedShuffleSplit
data = featureFormat(my_dataset, features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)
cv = StratifiedShuffleSplit(labels, 1000, random_state = 42)
```

6.
An evaluation metric is used to evaluate the performance of the model that is created. In this case, we have accuracy, recall and precision as the three most important metric that is used to test the model. Accuracy is the number of predictions we got right meaning it is the sum of true positives and true negatives over the whole data set. Recall is the number of correct true predicitons that the model made out of all possible true results which is true positives divided by the sum of true positive and false negatives. Precision is the number of correct true predictions out of all positive predictions that the model made which is true positives divided by the sum of true positive and false positive. In this case, higher recall means that my model is good to point out the POIs out of all of the POIs involved in the fraud while higher precision means that when my model say someone is a POI, there is a high chance that he is actually a POI. There are other metrics calculated which is f1 score and f2 score which is some combination of recall and precision. Using my model, I obtained slightly above 0.3 for precision and recall.

``` python
true_negatives = 0
false_negatives = 0
true_positives = 0
false_positives = 0
for train_idx, test_idx in cv: 
    features_train = []
    features_test  = []
    labels_train   = []
    labels_test    = []
    for ii in train_idx:
        features_train.append( features[ii] )
        labels_train.append( labels[ii] )
    for jj in test_idx:
        features_test.append( features[jj] )
        labels_test.append( labels[jj] )

    clf.fit(features_train, labels_train)

    predictions = clf.predict(features_test)

    for prediction, truth in zip(predictions, labels_test):
        if prediction == 0 and truth == 0:
            true_negatives += 1
        elif prediction == 0 and truth == 1:
            false_negatives += 1
        elif prediction == 1 and truth == 0:
            false_positives += 1
        elif prediction == 1 and truth == 1:
            true_positives += 1
        else:
            print "Evaluating performance for processed predictions:"
            break
try:
    print true_negatives, false_negatives, false_positives, true_positives
    total_predictions = true_negatives + false_negatives + false_positives + true_positives
    accuracy = 1.0*(true_positives + true_negatives)/total_predictions
    precision = 1.0*true_positives/(true_positives+false_positives)
    recall = 1.0*true_positives/(true_positives+false_negatives)
    print accuracy,precision,recall
```

